In [216]:
import pandas as pd
import numpy as np

In [217]:
pd.set_option('display.width', None)


In [218]:
df = pd.read_csv('Data Engineer Take home dataset - dataset.csv')

In [219]:
df.head()

,timestamp,user_id,event,flyer_id,merchant_id
0,2018-10-01T13:54:59-04:00,9ea672779feb1e088caa593b61ec51440c1d8694e1bf3dfe95554f7c9698c47e,shopping_list_open,NaN,NaN
1,2018-10-01T13:34:04-04:00,01ca5536abc5e0992c8e12ab57e93f567cb6608c6ebc1f403891b78a5fe75459,shopping_list_open,NaN,NaN
2,2018-10-01T13:56:36-04:00,1c1231e7a41a1bee17dd8e8111ebaef941525995f330199844fd9a0293edf9aa,flyer_open,2016315,2268
3,2018-10-01T13:36:16-04:00,b38f97880ccb69e8f89201eedbccc149a9ea914ca86111340a9b1d9de1788a01,shopping_list_open,NaN,NaN
4,2018-10-01T13:56:10-04:00,b38f97880ccb69e8f89201eedbccc149a9ea914ca86111340a9b1d9de1788a01,shopping_list_open,NaN,NaN


# Assumption

It is not given in the problem statement and neither in the dataset that how much time a user spends on a particular event i.e. if a user triggers a `flyer_open` event at `2018-10-01T10:00:00-04:00` for a particular flyer_id and then a new event `item_open` get registered at `2018-10-01T10:30:00-04:00` then does that mean that the user was in the flyer_open event for 30 minutes frmo 10:00 am to 10:30 am?


Since this information is not clear i have considered each event to span for 1 second,  i.e. when a `flyer_open` event occurs it takes place for 1 second and when another event `item_open` gets trigeered by the same user for the same flyer then both the event spans for 1 seconds each and thus cumulitavley user spends 2 seconds on that particular flyer. 

So the average time per user is time spent by user in seconds divided by number of flyers the user interacted with.

In [220]:
df.dropna(inplace=True)

In [221]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [222]:
time_spent_per_user = df.groupby(['user_id']).size().reset_index(name='Time Spent (in seconds)')

In [223]:
unique_flyer_per_user = df.groupby(['user_id'])["flyer_id"].unique().apply(len).reset_index(name='unique_flyers')
unique_flyer_per_user

,user_id,unique_flyers
0,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf937fa6ed729e66a13f91,20
1,002c17c9db976a1cad3516033b5023c5e11eff08fdbcb151bfa2d3c1b2250e27,7
2,00a52eb26d5b84881fea69429e91d2e7e7cb1004db83f867a1a38214da5def41,3
3,00d6164ad0d97a07a8ff2fbc6e10352736e4a8c02be283731a20cb261eea4c46,4
4,00dbcd0f49522e8614a0e363293b2a371f4083919b5d276524ce7d985970f3de,1
...,...,...
1161,ff67491a33ef563599366567f3176a3c12de59edb47b8081aa6761df9c336bc2,2
1162,ff7a676339f62ad51b0ae5c76a4d7623a157a13cf0c2f7bf3212f0bd462c43b9,25
1163,ffc050c996651a359d302abb80e76fbe548185c98aeda37662d674d33559e476,4
1164,ffd86b22f4cc4f7a28b49e3b8bda03d6ba2be51980c6b5aa688952caeafee6ec,2


In [224]:
result = pd.merge(time_spent_per_user, unique_flyer_per_user, on='user_id')

In [225]:
result["Avg. Time spent in seconds"] = result["Time Spent (in seconds)"] / result["unique_flyers"]

In [227]:
result.head()

,user_id,Time Spent (in seconds),unique_flyers,Avg. Time spent in seconds
0,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf937fa6ed729e66a13f91,24,20,1.2
1,002c17c9db976a1cad3516033b5023c5e11eff08fdbcb151bfa2d3c1b2250e27,7,7,1.0
2,00a52eb26d5b84881fea69429e91d2e7e7cb1004db83f867a1a38214da5def41,3,3,1.0
3,00d6164ad0d97a07a8ff2fbc6e10352736e4a8c02be283731a20cb261eea4c46,4,4,1.0
4,00dbcd0f49522e8614a0e363293b2a371f4083919b5d276524ce7d985970f3de,1,1,1.0


In [231]:
%timeit

def calculate_average_time_per_user(df: pd.DataFrame):
    df.dropna(inplace=True)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    time_spent_per_user = df.groupby(['user_id']).size().reset_index(name='Time Spent (in seconds)')
    unique_flyer_per_user = df.groupby(['user_id'])["flyer_id"].unique().apply(len).reset_index(name='unique_flyers')
    result = pd.merge(time_spent_per_user, unique_flyer_per_user, on='user_id')
    result["Avg. Time spent in seconds"] = result["Time Spent (in seconds)"] / result["unique_flyers"]
#     print(result.head())

    return result



In [236]:
%%time

calculate_average_time_per_user(df)



CPU times: user 44.2 ms, sys: 1.92 ms, total: 46.1 ms
Wall time: 45.4 ms


,user_id,Time Spent (in seconds),unique_flyers,Avg. Time spent in seconds
0,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf937fa6ed729e66a13f91,24,20,1.20
1,002c17c9db976a1cad3516033b5023c5e11eff08fdbcb151bfa2d3c1b2250e27,7,7,1.00
2,00a52eb26d5b84881fea69429e91d2e7e7cb1004db83f867a1a38214da5def41,3,3,1.00
3,00d6164ad0d97a07a8ff2fbc6e10352736e4a8c02be283731a20cb261eea4c46,4,4,1.00
4,00dbcd0f49522e8614a0e363293b2a371f4083919b5d276524ce7d985970f3de,1,1,1.00
...,...,...,...,...
1161,ff67491a33ef563599366567f3176a3c12de59edb47b8081aa6761df9c336bc2,2,2,1.00
1162,ff7a676339f62ad51b0ae5c76a4d7623a157a13cf0c2f7bf3212f0bd462c43b9,28,25,1.12
1163,ffc050c996651a359d302abb80e76fbe548185c98aeda37662d674d33559e476,4,4,1.00
1164,ffd86b22f4cc4f7a28b49e3b8bda03d6ba2be51980c6b5aa688952caeafee6ec,2,2,1.00
